## Model Evaluation


### Evaluate model


## Purpose:
This notebook evaluates the performance of the trained LinearRegression model. It includes:
- Loading the test data and trained model.
- Computing evaluation metrics (R², MAPE, etc.).
- Analyzing residuals and feature importance.
- Summarizing the model's effectiveness and next steps.

Imports and setup


In [3]:
import os
import pandas as pd
import joblib
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns

# Define paths
model_path = "../models/revenue_model.pkl"
features_path = "../data/processed/features.csv"
revenue_path = "../data/processed/reported_cleaned.csv"
output_path = "../reports/"

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)


## Load Data and Model

In [5]:
# Load features dataset
features = pd.read_csv(features_path)

# Load revenue dataset
revenue = pd.read_csv(revenue_path)

# Ensure period format matches between features and revenue
features["quarter"] = features["quarter"].str.replace(" ", "", regex=False)
revenue["period"] = revenue["period"].str.replace(" ", "", regex=False)

# Merge datasets
test_data = pd.merge(features, revenue, left_on="quarter", right_on="period", how="inner")

# Drop unnecessary columns
test_data = test_data.drop(columns=["quarter", "start_date", "end_date"], errors="ignore")
test_data.head()



,total_orders,total_spend_index,weekly_active_users_index,spend_per_user,period,revenue_index
0,12,1.020328,2.212705,0.461122,2018Q1,100.000000
1,25,3.907097,5.081092,0.768948,2018Q2,253.521127
2,23,9.063225,12.991272,0.697640,2018Q3,139.788732
3,37,15.179657,19.689239,0.770962,2018Q4,306.161972
4,25,13.636270,25.006123,0.545317,2019Q1,117.693662


In [7]:
# Load the trained model
if not os.path.exists(model_path):
    raise FileNotFoundError(f"The model file {model_path} does not exist.")
    
model = joblib.load(model_path)


## Evaluate model

In [8]:
# Features used during training
training_features = ["total_orders", "total_spend_index", "weekly_active_users_index", "spend_per_user"]

# Check for missing features
missing_features = [f for f in training_features if f not in test_data.columns]
if missing_features:
    raise ValueError(f"Missing features in the test dataset: {missing_features}")

# Define X_test and y_test
X_test = test_data[training_features]
y_test = test_data["revenue_index"]

# Predictions
y_pred = model.predict(X_test)

# Evaluation Metrics
metrics = {
    "R2 Score": r2_score(y_test, y_pred),
    "Mean Absolute Error": mean_absolute_error(y_test, y_pred),
    "Mean Squared Error": mean_squared_error(y_test, y_pred),
    "Mean Absolute Percentage Error": mean_absolute_percentage_error(y_test, y_pred)
}

# Display metrics
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")



R2 Score: 0.7952
Mean Absolute Error: 50.7357
Mean Squared Error: 3604.5861
Mean Absolute Percentage Error: 0.2015


## Feature Importance

In [10]:
# Extract model coefficients
coefficients = pd.DataFrame({
    "Feature": training_features,
    "Coefficient": model.coef_
})

# Display coefficients
print(coefficients)




                     Feature  Coefficient
0               total_orders     0.668272
1          total_spend_index     0.777118
2  weekly_active_users_index    -1.739244
3             spend_per_user   759.044918


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_random_forest)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
